In [ ]:
test_results = {
    'output_aggregation': {},
    'late_fusion_mlp': {},
    'late_fusion_pooling': {},
    'early_fusion': {},
    '3D_CNN': {},
    'dual_stream': {},
}

for category in test_results:
    test_results[category] = {
        'with_leakage': {},
        'without_leakage': {}
    }

# 1. With leakage

## 1.1. Load the dataset

In [ ]:
from utils.globalConst import *

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torchvision import transforms as T
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

from DatasetLoader.FrameImageDataset import FrameImageDataset
from DatasetLoader.FrameVideoDataset import FrameVideoDataset

train_transform = T.Compose([
    T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(5),
    T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    T.ToTensor(),
])

val_test_transform = T.Compose([
    T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    T.ToTensor(),
])

- - -
## 1.2. Train the models

In [ ]:
import torch.optim as optim
import torch.nn as nn

import json

### 1.2.1. Output aggregation

In [ ]:
frameimage_train_loader = DataLoader(
    dataset=FrameImageDataset(split='train', transform=train_transform, leakage=True), 
    batch_size=BATCH_SIZE, 
    shuffle=True
)
frameimage_val_loader = DataLoader(
    dataset=FrameImageDataset(split='val', transform=val_test_transform, leakage=True), 
    batch_size=BATCH_SIZE, 
    shuffle=True
)
frameimage_test_loader = DataLoader(
    dataset=FrameImageDataset(split='test', transform=val_test_transform, leakage=True), 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

In [ ]:
framevideolist_train_loader = DataLoader(
    dataset=FrameVideoDataset(split='train', transform=train_transform, leakage=True, stack_frames=False), 
    batch_size=BATCH_SIZE, 
    shuffle=True
)
framevideolist_val_loader = DataLoader(
    dataset=FrameVideoDataset(split='val', transform=val_test_transform, leakage=True, stack_frames=False), 
    batch_size=BATCH_SIZE, 
    shuffle=True
)
framevideolist_test_loader = DataLoader(
    dataset=FrameVideoDataset(split='test', transform=val_test_transform, leakage=True, stack_frames=False), 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

In [ ]:
from Models.OutputAggregation import OutputAggregation

model = OutputAggregation(device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

model.train_(
    num_epochs=5,
    optimizer=optimizer,
    scheduler=ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5),
    criterion=nn.CrossEntropyLoss(),
    train_loader=frameimage_train_loader,
    val_loader=frameimage_val_loader,
    plot=True
)

test_results['output_aggregation']['with_leakage'] = model.eval_(
    criterion=nn.CrossEntropyLoss(),
    test_loader=frameimage_test_loader
)

with open('results.json', 'w') as f:
    json.dump(test_results, f, indent=4)

### 1.2.2. Late fusion with MLP

In [ ]:
from Models.LateFusionMLP import LateFusionMLP

model = LateFusionMLP(device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

model.train_(
    num_epochs=5,
    optimizer=optimizer,
    scheduler=ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5),
    criterion=nn.CrossEntropyLoss(),
    train_loader=framevideolist_train_loader,
    val_loader=framevideolist_val_loader,
    plot=True
)

test_results['late_fusion_mlp']['with_leakage'] = model.eval_(
    criterion=nn.CrossEntropyLoss(),
    test_loader=framevideolist_test_loader
)

with open('results.json', 'w') as f:
    json.dump(test_results, f, indent=4)

### 1.2.3. Late fusion with Pooling

In [ ]:
from Models.LateFusionPooling import LateFusionPooling
from torch.optim.lr_scheduler import ReduceLROnPlateau

model = LateFusionPooling(device).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

model.train_(
    num_epochs=5,
    optimizer=optimizer,
    scheduler=ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True),
    criterion=nn.CrossEntropyLoss(),
    train_loader=framevideolist_train_loader,
    val_loader=framevideolist_val_loader,
    plot=True
)

test_results['late_fusion_pooling']['with_leakage'] = model.eval_(
    criterion=nn.CrossEntropyLoss(),
    test_loader=framevideolist_test_loader
)

with open('results.json', 'w') as f:
    json.dump(test_results, f, indent=4)

### 1.2.4. Early fusion

In [ ]:
from Models.EarlyFusion import EarlyFusion

model = EarlyFusion(device).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

model.train_(
    num_epochs=5,
    optimizer=optimizer,
    scheduler=ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True),
    criterion=nn.CrossEntropyLoss(),
    train_loader=framevideolist_train_loader,
    val_loader=framevideolist_val_loader,
    plot=True
)

test_results['early_fusion']['with_leakage'] = model.eval_(
    criterion=nn.CrossEntropyLoss(),
    test_loader=framevideolist_test_loader
)

with open('results.json', 'w') as f:
    json.dump(test_results, f, indent=4)

### 1.2.5. 3D CNN

In [ ]:
from Models.CNN3D import CNN3D

model = CNN3D(device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

model.train_(
    num_epochs=5,
    optimizer=optimizer,
    scheduler=ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True),
    criterion=nn.CrossEntropyLoss(),
    train_loader=framevideolist_train_loader,
    val_loader=framevideolist_val_loader,
    plot=True
)

test_results['3D_CNN']['with_leakage'] = model.eval_(
    criterion=nn.CrossEntropyLoss(),
    test_loader=framevideolist_test_loader
)

with open('results.json', 'w') as f:
    json.dump(test_results, f, indent=4)

- - -
- - -
# 2. Without leakage

## 2.1. Load the dataset

In [ ]:
frameimage_train_loader_no_leakage = DataLoader(
    dataset=FrameImageDataset(split='train', transform=train_transform, leakage=False), 
    batch_size=BATCH_SIZE, 
    shuffle=True
)
frameimage_val_loader_no_leakage = DataLoader(
    dataset=FrameImageDataset(split='val', transform=val_test_transform, leakage=False), 
    batch_size=BATCH_SIZE, 
    shuffle=True
)
frameimage_test_loader_no_leakage = DataLoader(
    dataset=FrameImageDataset(split='test', transform=val_test_transform, leakage=False), 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

In [ ]:
framevideolist_train_loader_no_leakage = DataLoader(
    dataset=FrameVideoDataset(split='train', transform=train_transform, leakage=False, stack_frames=False), 
    batch_size=BATCH_SIZE, 
    shuffle=True
)
framevideolist_val_loader_no_leakage = DataLoader(
    dataset=FrameVideoDataset(split='val', transform=val_test_transform, leakage=False, stack_frames=False), 
    batch_size=BATCH_SIZE, 
    shuffle=True
)
framevideolist_test_loader_no_leakage = DataLoader(
    dataset=FrameVideoDataset(split='test', transform=val_test_transform, leakage=False, stack_frames=False), 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

## 2.2. Train the models

In [ ]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

### 2.2.1. Output aggregation

In [ ]:
from Models.OutputAggregation import OutputAggregation
from torch.optim.lr_scheduler import ReduceLROnPlateau

model = OutputAggregation(device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

model.train_(
    num_epochs=5,
    optimizer=optimizer,
    scheduler=ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5),
    criterion=criterion,
    train_loader=frameimage_train_loader_no_leakage,
    val_loader=frameimage_val_loader_no_leakage,
    plot=True
)

test_results['output_aggregation']['without_leakage'] = model.eval_( 
    criterion=criterion,
    test_loader=frameimage_test_loader_no_leakage
)

with open('results.json', 'w') as f:
    json.dump(test_results, f, indent=4)

### 2.2.2. Late fusion with MLP

In [ ]:
from Models.LateFusionMLP import LateFusionMLP

model = LateFusionMLP(device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

model.train_(
    num_epochs=5,
    optimizer=optimizer,
    scheduler=ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5),
    criterion=criterion,
    train_loader=framevideolist_train_loader_no_leakage,
    val_loader=framevideolist_val_loader_no_leakage,
    plot=True
)

test_results['late_fusion_mlp']['without_leakage'] = model.eval_(
    criterion=criterion,
    test_loader=framevideolist_test_loader_no_leakage
)

### 2.2.3. Late fusion with Pooling

In [ ]:
from Models.LateFusionPooling import LateFusionPooling
from torch.optim.lr_scheduler import ReduceLROnPlateau

model = LateFusionPooling(device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

model.train_(
    num_epochs=5,
    optimizer=optimizer,
    scheduler=ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5),
    criterion=criterion,
    train_loader=framevideolist_train_loader_no_leakage,
    val_loader=framevideolist_val_loader_no_leakage,
    plot=True
)

test_results['late_fusion_pooling']['without_leakage'] = model.eval_(
    criterion=criterion,
    test_loader=framevideolist_test_loader_no_leakage
)

### 2.2.4. Early fusion

In [ ]:
from Models.EarlyFusion import EarlyFusion

model = EarlyFusion(device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

model.train_(
    num_epochs=5,
    optimizer=optimizer,
    scheduler=ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5),
    criterion=nn.CrossEntropyLoss(label_smoothing=0.1),
    train_loader=framevideolist_train_loader_no_leakage,
    val_loader=framevideolist_val_loader_no_leakage,
    plot=True
)

test_results['early_fusion']['without_leakage'] = model.eval_(
    criterion=nn.CrossEntropyLoss(label_smoothing=0.1),
    test_loader=framevideolist_test_loader_no_leakage
)

### 2.2.5. 3D CNN

In [ ]:
from Models.CNN3D import CNN3D

model = CNN3D(device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

model.train_(
    num_epochs=5,
    optimizer=optimizer,
    scheduler=ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5),
    criterion=nn.CrossEntropyLoss(label_smoothing=0.1),
    train_loader=framevideolist_train_loader_no_leakage,
    val_loader=framevideolist_val_loader_no_leakage,
    plot=True
)

test_results['3D_CNN']['without_leakage'] = model.eval_(
    criterion=nn.CrossEntropyLoss(label_smoothing=0.1),
    test_loader=framevideolist_test_loader_no_leakage
)

- - -
- - -
# 3. Dual Stream model

In [ ]:
flow_transform = T.Compose([
    T.Resize((IMAGE_SIZE, IMAGE_SIZE)),                   
    T.Normalize(mean=[0.5] * 2, std=[0.5] * 2),
])

In [ ]:
from DatasetLoader.FlowVideoDataset import FlowVideoDataset

flowvideo_train_loader = DataLoader(
    dataset=FlowVideoDataset(split='train', transform=flow_transform), 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

flowvideo_val_loader = DataLoader(
    dataset=FlowVideoDataset(split='val', transform=flow_transform), 
    batch_size=BATCH_SIZE, 
    shuffle=False
)

flowvideo_test_loader = DataLoader(
    dataset=FlowVideoDataset(split='test', transform=flow_transform), 
    batch_size=BATCH_SIZE, 
    shuffle=False
)

In [ ]:
from collections import Counter

train_labels = [label for _, label in frameimage_train_loader.dataset]
class_counts = Counter(train_labels)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(train_labels)
class_weights = compute_class_weight('balanced', classes=classes, y=train_labels)
print(f"Class Counts: {class_counts}")

In [ ]:
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
print(f"Class Weights: {class_weights}")

In [ ]:
from Models.DualStream import DualStream
model = DualStream(device).to(device)

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

In [ ]:
model.train_(
    num_epochs=5,
    optimizer=optimizer,
    scheduler=scheduler,
    criterion=criterion,
    train_loader=flowvideo_train_loader,
    val_loader=flowvideo_val_loader,
    plot=True
)

test_results['dual_stream']['without_leakage'] = model.eval_(
    criterion=criterion,
    test_loader=flowvideo_test_loader
)